In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"✅ Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

In [2]:
import uuid
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("Google imports happened")

Google imports happened


In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

In [ ]:
server_params = StdioServerParameters(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-everything"],
    tool_filter=["getTinyImage"],
)

mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=server_params,
        timeout=30,
    ))

In [ ]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

runner = InMemoryRunner(agent=image_agent)
response = await runner.run_debug("Provide a sample tiny image", verbose=True)

In [ ]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

So before we start this let me clarify so that everyone can understand how things work 

![Agent Workflow Diagram](../../resources/Untitled%20diagram-2025-11-14-073257.png)


In [4]:
# ============================================================================
# SEQUENCE 1: TOOL DEFINITION (Called by Agent during execution)
# ============================================================================

LARGE_ORDER_THRESHOLD = 5

def place_shipping_order(
    num_containers: int, 
    destination: str, 
    tool_context
) -> dict:
    """
    Tool that places shipping orders with approval for large orders.
    
    This function is called by the Agent (not by you directly).
    It implements three scenarios based on order size and call state.
    
    SEQUENCE IN EXECUTION:
    - First call with small order (≤5): Returns approved immediately
    - First call with large order (>5): Pauses and requests approval
    - Second call (resumed): Processes approval decision
    
    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination
        tool_context: Provided by ADK, contains pause/resume state
        
    Returns:
        Dictionary with order status and details
    """
    
    # -------------------------------------------------------------------------
    # SCENARIO 1: Small orders - Auto-approve immediately
    # -------------------------------------------------------------------------
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }
    
    # -------------------------------------------------------------------------
    # SCENARIO 2: Large order - FIRST CALL - Request approval and pause
    # -------------------------------------------------------------------------
    # Check if this is the first call (no confirmation exists yet)
    if not tool_context.tool_confirmation:
        # This call tells ADK to pause execution
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Approve this order?",
            payload={
                "num_containers": num_containers,
                "destination": destination,
                "timestamp": "2025-11-14"
            }
        )
        
        # Return immediately - execution will pause here
        return {
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }
    
    # -------------------------------------------------------------------------
    # SCENARIO 3: Large order - RESUMED CALL - Process approval decision
    # -------------------------------------------------------------------------
    # At this point, tool_context.tool_confirmation exists (we're resuming)
    
    # Retrieve saved data from the pause
    saved_data = tool_context.tool_confirmation.payload
    
    # Check human's decision
    if tool_context.tool_confirmation.confirmed:
        # APPROVED
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved by manager: {num_containers} containers to {destination}",
        }
    else:
        # REJECTED
        return {
            "status": "rejected",
            "num_containers": num_containers,
            "destination": destination,
            "reason": "Manager denied the request",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }



In [5]:
# ============================================================================
# SEQUENCE 2: AGENT & APP SETUP (Run once at startup)
# ============================================================================

def create_shipping_agent():
    """
    Creates the AI agent with the shipping tool.
    
    The agent's job:
    - Understand user's shipping requests
    - Decide when to use the place_shipping_order tool
    - Format responses to the user
    
    The agent does NOT handle pause/resume logic!
    """
    agent = LlmAgent(
        name="shipping_agent",
        model=Gemini(model="gemini-2.0-flash-exp"),
        instruction="""You are a shipping coordinator assistant.
  
When users request to ship containers:
1. Use the place_shipping_order tool with the number of containers and destination
2. If the order status is 'pending', inform the user that approval is required
3. After receiving the final result, provide a clear summary including:
   - Order status (approved/rejected)
   - Order ID (if available)
   - Number of containers and destination
4. Keep responses concise but informative
        """,
        tools=[FunctionTool(func=place_shipping_order)],
    )
    return agent


def create_shipping_app(agent):
    """
    Wraps the agent in a resumable app.
    
    This is CRITICAL for LRO (Long-Running Operations).
    Without ResumabilityConfig, the agent cannot pause/resume.
    
    Args:
        agent: The LlmAgent created by create_shipping_agent()
        
    Returns:
        App configured for pause/resume functionality
    """
    app = App(
        name="shipping_coordinator",
        root_agent=agent,
        resumability_config=ResumabilityConfig(is_resumable=True)  # Enables LRO
    )
    return app


def create_runner(app):
    """
    Creates the runner that executes the app.
    
    The runner is the entry point for all agent interactions.
    It manages sessions and routes requests to ADK.
    
    Args:
        app: The App created by create_shipping_app()
        
    Returns:
        Tuple of (runner, session_service)
    """
    session_service = InMemorySessionService()
    runner = Runner(
        app=app,
        session_service=session_service
    )
    return runner, session_service


In [7]:
# ============================================================================
# SEQUENCE 3: HELPER FUNCTIONS (Called by workflow)
# ============================================================================

def find_pause_event(events):
    """
    Searches through events to find if the agent paused.
    
    ADK automatically creates an 'adk_request_confirmation' event
    when a tool calls request_confirmation().
    
    This function searches for that special event.
    
    CALLED BY: run_shipping_workflow (after first agent call)
    
    Args:
        events: List of events returned by runner.run_async()
        
    Returns:
        Dictionary with pause details if found: 
            {"approval_id": str, "invocation_id": str}
        None if no pause event found
    """
    for event in events:
        # Check if this event has content
        if event.content and event.content.parts:
            # Loop through all parts of the event
            for part in event.content.parts:
                # Check if this part is a function call
                if part.function_call:
                    # Check if it's the special pause event
                    if part.function_call.name == "adk_request_confirmation":
                        # Found it! Extract the IDs needed for resuming
                        return {
                            "approval_id": part.function_call.id,
                            "invocation_id": event.invocation_id
                        }
    
    # No pause event found
    return None


def extract_text_from_events(events):
    """
    Extracts all text responses from events.
    
    Events can contain multiple parts (text, tool calls, etc.).
    This function extracts just the text parts for display.
    
    CALLED BY: run_shipping_workflow (to display agent responses)
    
    Args:
        events: List of events from runner.run_async()
        
    Returns:
        List of text strings
    """
    texts = []
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    texts.append(part.text)
    return texts


def display_agent_responses(events):
    """
    Prints agent's text responses to the console.
    
    CALLED BY: run_shipping_workflow (to show agent output)
    
    Args:
        events: List of events from runner.run_async()
    """
    texts = extract_text_from_events(events)
    for text in texts:
        print(f"Agent > {text}")


def create_approval_message(pause_info, user_approved):
    """
    Creates the approval response message to resume the agent.
    
    This message tells ADK:
    - Which approval request we're responding to (approval_id)
    - What the human decided (confirmed: True/False)
    
    CALLED BY: run_shipping_workflow (after getting human decision)
    
    Args:
        pause_info: Dictionary from find_pause_event() 
                   containing approval_id and invocation_id
        user_approved: Boolean - True if approved, False if rejected
        
    Returns:
        types.Content message formatted for ADK
    """
    # Create the function response that ADK expects
    confirmation_response = types.FunctionResponse(
        id=pause_info["approval_id"],  # Must match the pause event's ID
        name="adk_request_confirmation",  # Must be this exact name
        response={"confirmed": user_approved}  # The human's decision
    )
    
    # Wrap in Content object
    message = types.Content(
        role="user",
        parts=[types.Part(function_response=confirmation_response)]
    )
    
    return message


def get_human_approval_decision():
    """
    Gets approval decision from the human user via console input.
    
    CALLED BY: run_shipping_workflow (when agent pauses)
    
    Returns:
        Boolean - True if approved, False if rejected
    """
    while True:
        print("\n" + "="*60)
        print("⏸️  EXECUTION PAUSED - APPROVAL REQUIRED")
        print("="*60)
        decision = input("Do you approve this order? (Y/N): ").strip().upper()
        
        if decision == 'Y':
            print("✅ You approved the order")
            return True
        elif decision == 'N':
            print("❌ You rejected the order")
            return False
        else:
            print("❌ Invalid input. Please enter Y or N.")


def create_new_session(app_name, user_id):
    """
    Creates a new session for the conversation.
    
    A session is like a conversation thread that stores history.
    Each workflow run creates a new session.
    
    CALLED BY: run_shipping_workflow (at the start)
    
    Args:
        app_name: Name of the application
        user_id: ID of the user
        
    Returns:
        Unique session_id string
    """
    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"
    
    # Tell ADK to create the session
    # Note: In async context, this needs await
    # For simplicity, we'll return the ID and create it in the async function
    return session_id


def format_user_query(query_text):
    """
    Converts plain text query into ADK message format.
    
    CALLED BY: run_shipping_workflow (before sending to agent)
    
    Args:
        query_text: Plain text string from user
        
    Returns:
        types.Content object formatted for ADK
    """
    message = types.Content(
        role="user",
        parts=[types.Part(text=query_text)]
    )
    return message



In [8]:
# ============================================================================
# SEQUENCE 4: MAIN WORKFLOW (Entry point - orchestrates everything)
# ============================================================================

async def send_initial_request_to_agent(runner, user_id, session_id, user_message):
    """
    Sends the first request to the agent and collects all events.
    
    This is the FIRST call to run_async().
    ADK will create a new invocation_id internally.
    If the tool pauses, events will include the pause signal.
    
    SEQUENCE POSITION: Called first in workflow
    
    Args:
        runner: The Runner instance
        user_id: User identifier
        session_id: Session identifier
        user_message: Formatted user message (types.Content)
        
    Returns:
        List of events from the agent execution
    """
    print("\n🚀 Sending request to agent...\n")
    
    events = []
    
    # Call the agent - this might pause if order is large
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=user_message
    ):
        events.append(event)
    
    print("📦 Received response from agent")
    
    return events


async def resume_agent_with_approval(runner, user_id, session_id, approval_message, invocation_id):
    """
    Resumes the paused agent with the human's approval decision.
    
    This is the SECOND call to run_async().
    The key difference: we pass invocation_id parameter.
    This tells ADK to RESUME the previous execution, not start new.
    
    SEQUENCE POSITION: Called after pause is detected and approval obtained
    
    Args:
        runner: The Runner instance
        user_id: User identifier
        session_id: Session identifier (same as first call)
        approval_message: Formatted approval response
        invocation_id: The ID from the pause event (tells ADK to resume)
        
    Returns:
        List of events from the resumed execution
    """
    print("\n🔄 Resuming agent with approval decision...\n")
    
    events = []
    
    # Call the agent again - WITH invocation_id to resume
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=approval_message,
        invocation_id=invocation_id  # CRITICAL: This makes it a RESUME
    ):
        events.append(event)
    
    print("📦 Received final response from agent")
    
    return events


async def run_shipping_workflow(runner, session_service, query_text):
    """
    Main workflow function that orchestrates the entire LRO flow.
    
    EXECUTION SEQUENCE:
    1. Setup: Create session, format message
    2. Send initial request to agent
    3. Check if agent paused (look for pause event)
    4. If paused:
       a. Get human approval decision
       b. Resume agent with decision
       c. Display final result
    5. If not paused:
       a. Display result immediately
    
    This is the ENTRY POINT - call this function to start a shipping request.
    
    Args:
        runner: The Runner instance (from create_runner)
        session_service: The session service instance
        query_text: User's query as plain text string
    """
    
    # -------------------------------------------------------------------------
    # PHASE 1: SETUP
    # -------------------------------------------------------------------------
    print("\n" + "="*70)
    print("📦 SHIPPING ORDER WORKFLOW")
    print("="*70)
    print(f"User Query: {query_text}")
    
    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"
    user_id = "test_user"
    app_name = "shipping_coordinator"
    
    # Create session in the service
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id
    )
    print(f"📝 Session created: {session_id}")
    
    # Format user query into proper message format
    user_message = format_user_query(query_text)
    
    # -------------------------------------------------------------------------
    # PHASE 2: INITIAL AGENT CALL
    # -------------------------------------------------------------------------
    # Send request to agent - might pause if large order
    initial_events = await send_initial_request_to_agent(
        runner=runner,
        user_id=user_id,
        session_id=session_id,
        user_message=user_message
    )
    
    # -------------------------------------------------------------------------
    # PHASE 3: CHECK FOR PAUSE
    # -------------------------------------------------------------------------
    # Search for the special pause event
    pause_info = find_pause_event(initial_events)
    
    # -------------------------------------------------------------------------
    # PHASE 4: HANDLE RESULT BASED ON PAUSE STATUS
    # -------------------------------------------------------------------------
    
    if pause_info:
        # ═════════════════════════════════════════════════════════════════
        # PATH A: AGENT PAUSED - Need approval and resume
        # ═════════════════════════════════════════════════════════════════
        
        print("\n⏸️  Agent paused - approval required")
        print(f"🔑 Pause detected:")
        print(f"   - Approval ID: {pause_info['approval_id']}")
        print(f"   - Invocation ID: {pause_info['invocation_id']}")
        
        # Display what agent said before pausing
        print("\n💬 Agent's message:")
        display_agent_responses(initial_events)
        
        # Get human's approval decision (Y/N)
        user_approved = get_human_approval_decision()
        
        # Create the approval response message
        approval_message = create_approval_message(pause_info, user_approved)
        
        # Resume the agent with the approval decision
        # CRITICAL: Pass the invocation_id to resume the same execution
        final_events = await resume_agent_with_approval(
            runner=runner,
            user_id=user_id,
            session_id=session_id,
            approval_message=approval_message,
            invocation_id=pause_info["invocation_id"]  # Resume key!
        )
        
        # Display final result
        print("\n✅ FINAL RESULT:")
        print("-" * 70)
        display_agent_responses(final_events)
        
    else:
        # ═════════════════════════════════════════════════════════════════
        # PATH B: NO PAUSE - Order completed immediately
        # ═════════════════════════════════════════════════════════════════
        
        print("\n✅ Order completed immediately (no approval needed)")
        print("-" * 70)
        display_agent_responses(initial_events)
    
    # -------------------------------------------------------------------------
    # COMPLETION
    # -------------------------------------------------------------------------
    print("="*70)
    print("✅ Workflow completed\n")


In [9]:
# ============================================================================
# SEQUENCE 5: INITIALIZATION & MAIN EXECUTION
# ============================================================================

async def main():
    """
    Main entry point - sets up the system and runs example workflows.
    
    EXECUTION ORDER:
    1. Create agent
    2. Create app (with resumability)
    3. Create runner
    4. Run example workflows
    """
    
    print("\n" + "="*70)
    print("🏗️  INITIALIZING SHIPPING SYSTEM")
    print("="*70)
    
    # Step 1: Create the agent
    print("1️⃣  Creating shipping agent...")
    agent = create_shipping_agent()
    print("   ✅ Agent created")
    
    # Step 2: Wrap agent in resumable app
    print("2️⃣  Creating resumable app...")
    app = create_shipping_app(agent)
    print("   ✅ App created with LRO capability")
    
    # Step 3: Create runner
    print("3️⃣  Creating runner...")
    runner, session_service = create_runner(app)
    print("   ✅ Runner ready")
    
    print("\n✅ System initialized successfully!\n")
    
    # -------------------------------------------------------------------------
    # Run example workflows
    # -------------------------------------------------------------------------
    
    # Example 1: Small order (auto-approve)
    print("\n" + "🔹"*35)
    print("EXAMPLE 1: Small order (should auto-approve)")
    print("🔹"*35)
    await run_shipping_workflow(runner, session_service, "Ship 3 containers to New York")
    
    # Example 2: Large order (requires approval)
    print("\n" + "🔹"*35)
    print("EXAMPLE 2: Large order (will require your approval)")
    print("🔹"*35)
    await run_shipping_workflow(runner, session_service, "Ship 10 containers to Rotterdam")
    
    # Example 3: Another large order
    print("\n" + "🔹"*35)
    print("EXAMPLE 3: Another large order")
    print("🔹"*35)
    await run_shipping_workflow(runner, session_service, "Ship 15 containers to Tokyo")


In [11]:
# ============================================================================
# RUN THE SYSTEM
# ============================================================================

if __name__ == "__main__":
    import asyncio
    
    print("""
    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║           SHIPPING ORDER SYSTEM WITH LRO APPROVAL                ║
    ║                                                                  ║
    ║  This system demonstrates Long-Running Operations (LRO)          ║
    ║  with human-in-the-loop approval.                                ║
    ║                                                                  ║
    ║  Rules:                                                          ║
    ║  • Orders ≤ 5 containers: Auto-approved                          ║
    ║  • Orders > 5 containers: Require approval                       ║
    ║                                                                  ║
    ║  You will be prompted to approve/reject large orders.            ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)
    
    # Run the async main function
    asyncio.run(main())


    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║           SHIPPING ORDER SYSTEM WITH LRO APPROVAL                ║
    ║                                                                  ║
    ║  This system demonstrates Long-Running Operations (LRO)          ║
    ║  with human-in-the-loop approval.                                ║
    ║                                                                  ║
    ║  Rules:                                                          ║
    ║  • Orders ≤ 5 containers: Auto-approved                          ║
    ║  • Orders > 5 containers: Require approval                       ║
    ║                                                                  ║
    ║  You will be prompted to approve/reject large orders.            ║
    ║                                                                  ║
    ╚═════════════════════════════════════════════

RuntimeError: asyncio.run() cannot be called from a running event loop

Now let me show you how LRU architecture works 

![LRU Architecture Diagram](../../resources/Untitled%20diagram-2025-11-14-075525.png)